<a href="https://colab.research.google.com/github/dede0702/Homer-vs-Bart-CNN/blob/main/Homer_vs_Bart_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
```python
# Introdução:
# Este código implementa um modelo de Classificação de Imagens usando uma Rede Neural Convolucional (CNN)
# para classificar personagens de desenhos animados (Homer e Bart Simpson). O código utiliza o framework PyTorch
# e uma arquitetura ResNet18 pré-treinada para um aprendizado mais eficiente. O dataset é carregado,
# pré-processado, e usado para treinar o modelo. Após o treinamento, o modelo é avaliado e salvo,
# e algumas predições são visualizadas.

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os


# Define o device (GPU se disponível, senão CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parâmetros
data_dir = "dataset_personagens" # Diretório do dataset
num_epochs = 25  # Número de épocas de treinamento
batch_size = 32 # Tamanho do batch
learning_rate = 0.001 # Taxa de aprendizado

# Transformações de dados: redimensiona, converte para tensor e normaliza as imagens
data_transforms = {
    'training_set': transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensiona as imagens para o tamanho exigido pela ResNet18
        transforms.ToTensor(), # Converte as imagens para tensores PyTorch
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normaliza as imagens usando média e desvio padrão do ImageNet
    ]),
    'test_set': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Carrega os datasets usando ImageFolder, que organiza as imagens por pasta (classe)
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['training_set', 'test_set']}

# Cria DataLoaders para carregar os dados em batches durante o treinamento
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) # shuffle=True embaralha os dados de treino
              for x in ['training_set', 'test_set']}

# Obtém o tamanho de cada dataset
dataset_sizes = {x: len(image_datasets[x]) for x in ['training_set', 'test_set']}
# Obtém os nomes das classes
class_names = image_datasets['training_set'].classes

# Definição do modelo: usa uma ResNet18 pré-treinada
model = torch.models.resnet18(pretrained=True) # Carrega o modelo ResNet18 pré-treinado no ImageNet
num_ftrs = model.fc.in_features # Obtém o número de features da penúltima camada
model.fc = nn.Linear(num_ftrs, len(class_names))  # Substitui a camada totalmente conectada final para corresponder ao número de classes do nosso dataset
model = model.to(device) # Move o modelo para o device (GPU ou CPU)


# Função de Treinamento
def train_model(model, criterion, optimizer, num_epochs=25):

    for epoch in range(num_epochs): # Loop pelas épocas
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['training_set', 'test_set']: # Loop pelas fases (treino e teste)
            if phase == 'training_set':
                model.train()  # Define o modelo para o modo de treinamento
            else:
                model.eval()   # Define o modelo para o modo de avaliação

            running_loss = 0.0 # Variável para acumular o loss da época
            running_corrects = 0 # Variável para acumular o número de predições corretas

            for inputs, labels in dataloaders[phase]: # Loop pelos batches de dados
                inputs = inputs.to(device) # Move os inputs para o device
                labels = labels.to(device) # Move os labels para o device

                optimizer.zero_grad() # Zera os gradientes do otimizador

                with torch.set_grad_enabled(phase == 'training_set'):  # Calcula os gradientes apenas durante o treino
                    outputs = model(inputs) # Passa os inputs pelo modelo
                    _, preds = torch.max(outputs, 1) # Obtém as predições
                    loss = criterion(outputs, labels) # Calcula o loss

                    if phase == 'training_set':
                        loss.backward() # Calcula os gradientes
                        optimizer.step() # Atualiza os pesos do modelo


                running_loss += loss.item() * inputs.size(0) # Acumula o loss
                running_corrects += torch.sum(preds == labels.data) # Acumula o número de predições corretas

            epoch_loss = running_loss / dataset_sizes[phase] # Calcula o loss médio da época
            epoch_acc = running_corrects.double() / dataset_sizes[phase] # Calcula a acurácia da época

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Define a função de perda (CrossEntropyLoss para classificação multiclasse)
criterion = nn.CrossEntropyLoss()
# Define o otimizador (Adam)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Treina o modelo
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)


# Salva o modelo treinado
torch.save(model.state_dict(), 'modelo_treinado.pth')

# Função para visualizar algumas predições do modelo
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test_set']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images // 2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)



def imshow(inp, title=None):
    """Imshow para Tensores."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated



visualize_model(model) # Visualiza as predições
plt.ioff()
plt.show()


# Conclusão:
# O código implementa um modelo de classificação de imagens utilizando uma CNN ResNet18 pré-treinada
# para classificar personagens de desenhos animados. A utilização de um modelo pré-treinado permite um
# treinamento mais rápido e eficiente. O código inclui etapas de pré-processamento de dados, treinamento,
# avaliação, salvamento do modelo e visualização de predições, fornecendo uma solução completa para
# o problema de classificação de imagens.
```


Algumas pequenas melhorias foram adicionadas aos comentários para maior clareza.  Este código agora está bem documentado e pronto para ser executado!